## Chapter 4 Exercises - Training Models


1. Which Linear Regression training algorithm can you use if you have a training set with millions of features? 
    - Gradient descent with lasso regression

2. Suppose the features in your training set have very different scales. Which algorithm might suffer from this and how? What can you do about it? 
    - Gradient descent will be slowed since it will preferentially follow the gradient of the feature with the larger scale first instead of taking a direct route to the minimum. 
    - feature scaling

3. Can Gradient Descent get stuck in a local minimum when training a logistic regression model? 
    - no, the cost function is convex

4. Do all gradient descent algorithms lead to the same model, provided you let them run long enough? 
    - if the learning rate is too large then an algorithm could fail to converge. 
    - if there are local minima in the cost function then one GD may get caught in the local minima while another algorithm may succesfully escape it and converge on the global mimimum or a different local minima. 
    
5. Suppose you use batch gradient descent and you plot the validation error at every epoch. If you notice the validation error consistently goes up, what is likely going on? How can you fix this? 
    - The model is getting over trained. 
    - Rollback to the model parameters at the minimum value for the validation error
    - use a simpler model
    - use a regularization technique
    - use a larger training set to reach a lower error value before overfitting occurs. 
    - If the learning rate is too high then the validation and training error will both go up.
    
6. Is it a good idea to stop mini-batch gradient descent immediately when the validation error goes up? 
    - no, in mini-batch gd the validation error will have some noise/variance from epoch to epoch. So you need to wait to see that it is consistently rising then roll back to the minimum. 
    
7. Which gradient descent alogrithm discussed will reach the viscinity of the optimal solution the fastest? Which will actually convege? How can you make others converge as well? 
    - Stochastic gradient descent will reach the right area fastest since it trains the fastest.
    - Batch gradient descent will converge on the actual optimal solution eventually
    - applying a good learning schedule to modulate the learning rate over the course of training can allow stochastic and mini-batch GD to converge. 
    
8. Suppose you are using polynomial regression. You plot the learning curves and you notice there is a large gap between the training error and the validation error. What is happening? What are three ways to solve this?
    - the gap means that the model is performing better on the training data vs the validation data. This means the model is overfitting the to the training data. 
    - You could use a simpler model such as linear regression or reduce the number of degress in your polynomial regression. 
    - You could regulate the cost function with ridge regression or lasso. Will reduce complexity by eliminating weights of unimportant features.  
    - You could provide a large training dataset to allow the model to train longer before it becomes over fit. 
    
9. Suppose you are using Ridge Regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter alpha or reduce it? 
    - the model has high bias since it seems to generalize well to the validation data but have poor performance. 
    - you should reduce alpha to trend more towards variance in the bias variance trade off. This will necessarily increase the variance, but if you manage to reduce the error enough before the variance rises to an unacceptable level then you may fix the problem. 
    
10. Why would you want to use:
    - Ridge Regression? Ridge regression will help to reduce the model's variance and generalize better to new data. It prevents overfitting.
    - Lasso instead of Ridge Regression? Lasso regression is useful if you suspect that many of your features are not important. Lasso will reduce the weights of unimportant features to zero or near zero. 
    - Elastic Net instead of Lasso?  If you have more features than instances in your training data then you should use elastic net
    
11. Suppose you want to classify pictures as outdoor/indoor and daytime/nightime. Should you implement two logistic regression classifiers or one softmax regression classifier? 
    - Since each picture will have up to two classes represented (daytime and outdoor or nightime and inside in the same pic) you should use two logistic regression classifiers. Although telling night from day indoors seems like it might be impossible any room without windows. 
    

##  12. Implement Batch Gradient Descent with early stopping for softmax regression without using scikitlearn. 

In [3]:
import numpy as np
import scipy as sp

In [9]:
X = 10 * np.random.rand(1000,4)
y = np.round(np.random.rand(1000,1))
X.shape

(1000, 4)

In [8]:
X_b = np.c_[np.ones((1000,1)), X]
X_b.shape

(1000, 5)

In [17]:
X_b_train = X_b[0:800,:]
X_b_val = X_b[800:1000,:]
y_train = y[0:800,:]
y_val = y[800:1000,:]

In [ ]:
#Batch GD
eta = 0.1 #learning rate
n_iterations = 1000
m =1000

theta = np.random.rand(4,1)

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta)-y)
    theta = theta - eta * gradients
    
#compute the linear regression prediction
#parameter vector for each class
#enter scores into softmax function
# use argmax to make softmax predicion
# cross entropy cost function
#early stopping 